In [2]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings(action = 'ignore')
warnings.filterwarnings('ignore')

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import re
import urllib.request
import json, csv
import math

In [2]:
cat4 = [63,56,54,55,60,53,52,61,57,58,65,64,68,66,69,59,62]
cat2 = [12,18,13,19,21,15,43,17,45,20,46,44,14,22]
cat3 = [70,71,72,23,28,24,50,33,47,32,25,31,48,27,26,34]
cat1 = [6,1,7,36,41,42,8,10,9,38,67,39,37,11]

In [1]:
category_type_level = ['밑반찬', '메인반찬', '국/탕', '찌개', '디저트', '면/만두', '밥/죽/떡', '퓨전', '김치/젓갈/장류', '양념/소스/햄', '양식', '샐러드', '스프', '빵', '과자', '차/음료/술', '기타']
category_situation_level = ['일상', '초스피드', '손님접대', '술안주', '다이어트', '도시락', '영양식', '간식', '야식', '푸드스타일링', '해장', '명절', '이유식', '기타']
category_ingredient_level = ['소고기', '돼지고기', '닭고기', '육류', '채소류', '해물류', '달걀/유제품', '가공식품류', '쌀', '밀가루', '건어물류', '버섯류', '과일류', '콩/견과류', '곡류', '기타']
category_method_level = ['볶음', '끓이기', '부침', '조림', '무침', '비빔', '찜', '절임', '튀김', '삶기', '굽기', '데치기', '회', '기타']

In [3]:
items = [category_type_level, category_situation_level, category_ingredient_level, category_method_level]

In [4]:
from itertools import product
item = list(product(*items))

In [5]:
item

[('밑반찬', '일상', '소고기', '볶음'),
 ('밑반찬', '일상', '소고기', '끓이기'),
 ('밑반찬', '일상', '소고기', '부침'),
 ('밑반찬', '일상', '소고기', '조림'),
 ('밑반찬', '일상', '소고기', '무침'),
 ('밑반찬', '일상', '소고기', '비빔'),
 ('밑반찬', '일상', '소고기', '찜'),
 ('밑반찬', '일상', '소고기', '절임'),
 ('밑반찬', '일상', '소고기', '튀김'),
 ('밑반찬', '일상', '소고기', '삶기'),
 ('밑반찬', '일상', '소고기', '굽기'),
 ('밑반찬', '일상', '소고기', '데치기'),
 ('밑반찬', '일상', '소고기', '회'),
 ('밑반찬', '일상', '소고기', '기타'),
 ('밑반찬', '일상', '돼지고기', '볶음'),
 ('밑반찬', '일상', '돼지고기', '끓이기'),
 ('밑반찬', '일상', '돼지고기', '부침'),
 ('밑반찬', '일상', '돼지고기', '조림'),
 ('밑반찬', '일상', '돼지고기', '무침'),
 ('밑반찬', '일상', '돼지고기', '비빔'),
 ('밑반찬', '일상', '돼지고기', '찜'),
 ('밑반찬', '일상', '돼지고기', '절임'),
 ('밑반찬', '일상', '돼지고기', '튀김'),
 ('밑반찬', '일상', '돼지고기', '삶기'),
 ('밑반찬', '일상', '돼지고기', '굽기'),
 ('밑반찬', '일상', '돼지고기', '데치기'),
 ('밑반찬', '일상', '돼지고기', '회'),
 ('밑반찬', '일상', '돼지고기', '기타'),
 ('밑반찬', '일상', '닭고기', '볶음'),
 ('밑반찬', '일상', '닭고기', '끓이기'),
 ('밑반찬', '일상', '닭고기', '부침'),
 ('밑반찬', '일상', '닭고기', '조림'),
 ('밑반찬', '일상', '닭고기', '무침'),
 ('밑반찬', '일상', '닭고기', '비빔'),

In [6]:
item_df = pd.DataFrame(item, columns = ['category_type_level', 'category_situation_level', 'category_ingredient_level', 'category_method_level'])

In [7]:
item_df.to_csv('meta_category.csv', index=False)

In [7]:
base_url = 'https://www.10000recipe.com/recipe/list.html?q=&query=&cat1={}&cat2={}&cat3={}&cat4={}&fct=&order=reco&lastcate=cat1&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource='

all_urls = []

for c1 in cat1:
    for c2 in cat2:
        for c3 in cat3:
            for c4 in cat4:
                url = base_url.format(c1, c2, c3, c4)
                all_urls.append(url)

In [8]:
ai = all_urls[:5000]
bi = all_urls[5000:10000]
ci = all_urls[10000:15000]
di = all_urls[15000:20000]
ei = all_urls[20000:25000]
fi = all_urls[25000:30000]
gi = all_urls[30000:35000]
hi = all_urls[35000:40000]
ii = all_urls[40000:45000]
ji = all_urls[45000:]

# Meta Category

## PART 1

In [27]:
import urllib.request
from urllib.error import URLError
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

a_food_lists = []

for url in tqdm(ai, desc="Crawling Progress"):
    if url:
        a_food_list = []

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            req = urllib.request.Request(url, headers=headers)
            code = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(code, 'html.parser')
            tag_cont_element = soup.find('ul', class_='tag_cont')

            if tag_cont_element:
                li_tags = tag_cont_element.find_all('li')
                for li in li_tags:
                    a_tag = li.find('a')
                    if a_tag:
                        name = a_tag.get_text()
                        a_food_list.append(name)
                    else:
                        a_food_list.append(np.nan)
                a_food_lists.append(a_food_list)
            else:
                a_food_lists.append([])
        except URLError as e:
            print(f"An error occurred while fetching URL: {url}\nError: {e}")
            a_food_lists.append([])
    else:
        a_food_lists.append([])

Crawling Progress:  61%|████████████████████████████████████                       | 3059/5000 [10:04<03:01, 10.71it/s]

An error occurred while fetching URL: https://www.10000recipe.com/recipe/list.html?q=&query=&cat1=6&cat2=44&cat3=23&cat4=59&fct=&order=reco&lastcate=cat1&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource=
Error: <urlopen error [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다>


Crawling Progress: 100%|███████████████████████████████████████████████████████████| 5000/5000 [14:33<00:00,  5.72it/s]


In [65]:
a_data = []
for i in range(len(a_food_lists)):
    li = '  '.join(a_food_lists[i])
    a_data.append(li)

In [66]:
a_data[0]

'소고기볶음  소고기고추장볶음  소고기오이볶음  소고기가지볶음  고추장볶음  표고버섯볶음  볶음고추장  소보로  약고추장  고구마순볶음  고추볶음  고추장  김치볶음  깻잎볶음  꽈리고추볶음  도시락  들기름막국수  등심볶음  마늘쫑볶음  무나물  미역볶음  불고기  불고기볶음  살치살  새송이버섯볶음  애호박볶음  양념장  양송이버섯볶음  양지볶음'

In [67]:
a_food_df = pd.DataFrame(a_data, columns = ['food_name'])

In [68]:
a_food_df.to_csv('a_meta_category.csv', index=False)

In [12]:
a_food_df = pd.read_csv('a_meta_category.csv')

## PART 2

In [15]:
import urllib.request
from urllib.error import URLError
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm
import time
b_food_lists = []

for url in tqdm(bi, desc="Crawling Progress"):
    if url:
        b_food_list = []

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            req = urllib.request.Request(url, headers=headers)
            code = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(code, 'html.parser')
            tag_cont_element = soup.find('ul', class_='tag_cont')
            time.sleep(1)
            if tag_cont_element:
                li_tags = tag_cont_element.find_all('li')
                for li in li_tags:
                    a_tag = li.find('a')
                    if a_tag:
                        name = a_tag.get_text()
                        b_food_list.append(name)
                    else:
                        b_food_list.append(np.nan)
                b_food_lists.append(b_food_list)
            else:
                b_food_lists.append([])
        except URLError as e:
            print(f"An error occurred while fetching URL: {url}\nError: {e}")
            b_food_lists.append([])
    else:
        b_food_lists.append([])

Crawling Progress:  67%|█████████████████████████████████████▉                   | 3333/5000 [1:06:57<32:39,  1.18s/it]

An error occurred while fetching URL: https://www.10000recipe.com/recipe/list.html?q=&query=&cat1=7&cat2=13&cat3=25&cat4=55&fct=&order=reco&lastcate=cat1&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource=
Error: <urlopen error [Errno 2] No such file or directory>


Crawling Progress: 100%|█████████████████████████████████████████████████████████| 5000/5000 [1:39:54<00:00,  1.20s/it]


In [20]:
len(b_food_lists)

5000

In [17]:
b_data = []
for i in range(len(b_food_lists)):
    li = '  '.join(b_food_lists[i])
    b_data.append(li)

In [18]:
print(b_data[0])

계란탕  곤약국수  순두부국


In [19]:
b_food_df = pd.DataFrame(b_data, columns = ['food_name'])
b_food_df.to_csv('b_meta_category.csv', index=False)

## PART 3

In [23]:
import urllib.request
from urllib.error import URLError
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm
import time
c_food_lists = []

for url in tqdm(ci, desc="Crawling Progress"):
    if url:
        c_food_list = []

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            req = urllib.request.Request(url, headers=headers)
            code = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(code, 'html.parser')
            tag_cont_element = soup.find('ul', class_='tag_cont')
            time.sleep(1)
            if tag_cont_element:
                li_tags = tag_cont_element.find_all('li')
                for li in li_tags:
                    a_tag = li.find('a')
                    if a_tag:
                        name = a_tag.get_text()
                        c_food_list.append(name)
                    else:
                        c_food_list.append(np.nan)
                c_food_lists.append(c_food_list)
            else:
                c_food_lists.append([])
        except URLError as e:
            print(f"An error occurred while fetching URL: {url}\nError: {e}")
            c_food_lists.append([])
    else:
        c_food_lists.append([])

Crawling Progress: 100%|█████████████████████████████████████████████████████████| 5000/5000 [1:38:02<00:00,  1.18s/it]


In [25]:
c_data = []
for i in range(len(c_food_lists)):
    li = '  '.join(c_food_lists[i])
    c_data.append(li)

print(c_data[0])

In [28]:
c_food_df = pd.DataFrame(c_data, columns = ['food_name'])
c_food_df.to_csv('c_meta_category.csv', index=False)

## PART 4

In [29]:
import urllib.request
from urllib.error import URLError
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm
import time
d_food_lists = []

for url in tqdm(di, desc="Crawling Progress"):
    if url:
        d_food_list = []

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            req = urllib.request.Request(url, headers=headers)
            code = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(code, 'html.parser')
            tag_cont_element = soup.find('ul', class_='tag_cont')
            time.sleep(1)
            if tag_cont_element:
                li_tags = tag_cont_element.find_all('li')
                for li in li_tags:
                    a_tag = li.find('a')
                    if a_tag:
                        name = a_tag.get_text()
                        d_food_list.append(name)
                    else:
                        d_food_list.append(np.nan)
                d_food_lists.append(d_food_list)
            else:
                d_food_lists.append([])
        except URLError as e:
            print(f"An error occurred while fetching URL: {url}\nError: {e}")
            d_food_lists.append([])
    else:
        d_food_lists.append([])

Crawling Progress: 100%|█████████████████████████████████████████████████████████| 5000/5000 [1:38:40<00:00,  1.18s/it]


In [30]:
d_data = []
for i in range(len(d_food_lists)):
    li = '  '.join(d_food_lists[i])
    d_data.append(li)

print(d_data[0])

In [32]:
len(d_data)

5000

In [33]:
d_food_df = pd.DataFrame(d_data, columns = ['food_name'])
d_food_df.to_csv('d_meta_category.csv', index=False)

## PART 5

In [9]:
import urllib.request
from urllib.error import URLError
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

e_food_lists = []

for url in tqdm(ei, desc="Crawling Progress"):
    if url:
        e_food_list = []

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            req = urllib.request.Request(url, headers=headers)
            code = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(code, 'html.parser')
            tag_cont_element = soup.find('ul', class_='tag_cont')
            time.sleep(1)
            if tag_cont_element:
                li_tags = tag_cont_element.find_all('li')
                for li in li_tags:
                    a_tag = li.find('a')
                    if a_tag:
                        name = a_tag.get_text()
                        e_food_list.append(name)
                    else:
                        e_food_list.append(np.nan)
                e_food_lists.append(e_food_list)
            else:
                e_food_lists.append([])
        except URLError as e:
            print(f"An error occurred while fetching URL: {url}\nError: {e}")
            e_food_lists.append([])
    else:
        e_food_lists.append([])

Crawling Progress: 100%|█████████████████████████████████████████████████████████| 5000/5000 [1:38:45<00:00,  1.19s/it]


In [10]:
e_data = []
for i in range(len(e_food_lists)):
    li = '  '.join(e_food_lists[i])
    e_data.append(li)

print(e_data)

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '무화과샐러드  컵샐러드', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '나초', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '군만두  막국수  쟁반국수', '', '', '', '소스', '', '', '', '', '', '', '', '', '', '', '', '', '샐러드', '육회비빔밥', '', '', '', '', '곤약샐러드  소고기샐러드', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '비빔국수  비빔당면', '비빔밥  닭가슴살볶음밥  새싹비빔밥  주먹밥  카레밥', '', '고추장', '', '', '닭가슴살샐러드  치킨샐러드', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '샐러드', '', '', '', '', '', '', '샐러드파스타', '', '', '', '비빔국수  샐러드파스타  국수  비빔면  곤약국수  비빔당면  국수호박  라면  막국수  비빔냉면  쌀국수  열무비빔국수  우동샐러드  쫄면

In [11]:
e_food_df = pd.DataFrame(e_data, columns = ['food_name'])
e_food_df.to_csv('e_meta_category.csv', index=False)

## PART 6

In [12]:
import urllib.request
from urllib.error import URLError
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

f_food_lists = []

for url in tqdm(fi, desc="Crawling Progress"):
    if url:
        f_food_list = []

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            req = urllib.request.Request(url, headers=headers)
            code = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(code, 'html.parser')
            tag_cont_element = soup.find('ul', class_='tag_cont')
            time.sleep(1)
            if tag_cont_element:
                li_tags = tag_cont_element.find_all('li')
                for li in li_tags:
                    a_tag = li.find('a')
                    if a_tag:
                        name = a_tag.get_text()
                        f_food_list.append(name)
                    else:
                        f_food_list.append(np.nan)
                f_food_lists.append(f_food_list)
            else:
                f_food_lists.append([])
        except URLError as e:
            print(f"An error occurred while fetching URL: {url}\nError: {e}")
            f_food_lists.append([])
    else:
        f_food_lists.append([])

Crawling Progress: 100%|█████████████████████████████████████████████████████████| 5000/5000 [1:36:50<00:00,  1.16s/it]


In [13]:
f_data = []
for i in range(len(f_food_lists)):
    li = '  '.join(f_food_lists[i])
    f_data.append(li)

f_data

['',
 '',
 '',
 '떡케이크  설기  술빵  찐빵  호떡',
 '양갱',
 '',
 '옥수수  옥수수삶는법  찰옥수수',
 '',
 '',
 '',
 '',
 '볼  브라우니  찐빵',
 '딤섬',
 '떡  약밥',
 '',
 '',
 '',
 '',
 '',
 '',
 '찐빵  찜케이크  케이크  핫케이크  호빵',
 '양갱',
 '',
 '찜  호빵',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '등갈비찜  수육',
 '',
 '',
 '',
 '만두  김치만두',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '통닭',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '두부찜',
 '',
 '',
 '고구마경단  치즈감자',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '오징어순대',
 '바지락찜',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '계란찜',
 '',
 '',
 '',
 '달걀',
 '',
 '컵밥',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [14]:
f_food_df = pd.DataFrame(f_data, columns = ['food_name'])
f_food_df.to_csv('f_meta_category.csv', index=False)

## PART 7

In [15]:
import urllib.request
from urllib.error import URLError
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

g_food_lists = []

for url in tqdm(gi, desc="Crawling Progress"):
    if url:
        g_food_list = []

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            req = urllib.request.Request(url, headers=headers)
            code = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(code, 'html.parser')
            tag_cont_element = soup.find('ul', class_='tag_cont')
            time.sleep(1)
            if tag_cont_element:
                li_tags = tag_cont_element.find_all('li')
                for li in li_tags:
                    a_tag = li.find('a')
                    if a_tag:
                        name = a_tag.get_text()
                        g_food_list.append(name)
                    else:
                        g_food_list.append(np.nan)
                g_food_lists.append(g_food_list)
            else:
                g_food_lists.append([])
        except URLError as e:
            print(f"An error occurred while fetching URL: {url}\nError: {e}")
            g_food_lists.append([])
    else:
        g_food_lists.append([])

Crawling Progress:  30%|████████████████▉                                        | 1491/5000 [29:02<1:08:26,  1.17s/it]

An error occurred while fetching URL: https://www.10000recipe.com/recipe/list.html?q=&query=&cat1=9&cat2=19&cat3=48&cat4=61&fct=&order=reco&lastcate=cat1&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource=
Error: <urlopen error [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다>


Crawling Progress: 100%|█████████████████████████████████████████████████████████| 5000/5000 [1:37:05<00:00,  1.17s/it]


In [16]:
g_data = []
for i in range(len(g_food_lists)):
    li = '  '.join(g_food_lists[i])
    g_data.append(li)

print(g_data[0])

In [17]:
g_food_df = pd.DataFrame(g_data, columns = ['food_name'])
g_food_df.to_csv('g_meta_category.csv', index=False)

## PART 8

In [18]:
import urllib.request
from urllib.error import URLError
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

h_food_lists = []

for url in tqdm(hi, desc="Crawling Progress"):
    if url:
        h_food_list = []

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            req = urllib.request.Request(url, headers=headers)
            code = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(code, 'html.parser')
            tag_cont_element = soup.find('ul', class_='tag_cont')
            time.sleep(1)
            if tag_cont_element:
                li_tags = tag_cont_element.find_all('li')
                for li in li_tags:
                    a_tag = li.find('a')
                    if a_tag:
                        name = a_tag.get_text()
                        h_food_list.append(name)
                    else:
                        h_food_list.append(np.nan)
                h_food_lists.append(h_food_list)
            else:
                h_food_lists.append([])
        except URLError as e:
            print(f"An error occurred while fetching URL: {url}\nError: {e}")
            h_food_lists.append([])
    else:
        h_food_lists.append([])

Crawling Progress:  53%|███████████████████████████████▏                           | 2646/5000 [51:37<46:15,  1.18s/it]

An error occurred while fetching URL: https://www.10000recipe.com/recipe/list.html?q=&query=&cat1=38&cat2=14&cat3=50&cat4=57&fct=&order=reco&lastcate=cat1&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource=
Error: <urlopen error [Errno 2] No such file or directory>


Crawling Progress: 100%|█████████████████████████████████████████████████████████| 5000/5000 [1:37:53<00:00,  1.17s/it]


In [19]:
h_data = []
for i in range(len(h_food_lists)):
    li = '  '.join(h_food_lists[i])
    h_data.append(li)

print(h_data[0])

In [20]:
h_food_df = pd.DataFrame(h_data, columns = ['food_name'])
h_food_df.to_csv('h_meta_category.csv', index=False)

## PART 9

In [21]:
import urllib.request
from urllib.error import URLError
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

i_food_lists = []

for url in tqdm(ii, desc="Crawling Progress"):
    if url:
        i_food_list = []

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            req = urllib.request.Request(url, headers=headers)
            code = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(code, 'html.parser')
            tag_cont_element = soup.find('ul', class_='tag_cont')
            time.sleep(1)
            if tag_cont_element:
                li_tags = tag_cont_element.find_all('li')
                for li in li_tags:
                    a_tag = li.find('a')
                    if a_tag:
                        name = a_tag.get_text()
                        i_food_list.append(name)
                    else:
                        i_food_list.append(np.nan)
                i_food_lists.append(i_food_list)
            else:
                i_food_lists.append([])
        except URLError as e:
            print(f"An error occurred while fetching URL: {url}\nError: {e}")
            i_food_lists.append([])
    else:
        i_food_lists.append([])

Crawling Progress:  32%|██████████████████▍                                      | 1621/5000 [31:22<1:05:06,  1.16s/it]

An error occurred while fetching URL: https://www.10000recipe.com/recipe/list.html?q=&query=&cat1=67&cat2=22&cat3=70&cat4=53&fct=&order=reco&lastcate=cat1&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource=
Error: <urlopen error [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다>


Crawling Progress:  82%|██████████████████████████████████████████████▌          | 4081/5000 [1:18:37<17:23,  1.14s/it]

An error occurred while fetching URL: https://www.10000recipe.com/recipe/list.html?q=&query=&cat1=39&cat2=45&cat3=71&cat4=63&fct=&order=reco&lastcate=cat1&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource=
Error: <urlopen error [Errno 2] No such file or directory>


Crawling Progress: 100%|█████████████████████████████████████████████████████████| 5000/5000 [1:36:09<00:00,  1.15s/it]


In [22]:
i_data = []
for i in range(len(i_food_lists)):
    li = '  '.join(i_food_lists[i])
    i_data.append(li)

print(i_data[0])

꼬치구이  또띠아  불고기또띠아피자


In [23]:
i_food_df = pd.DataFrame(i_data, columns = ['food_name'])
i_food_df.to_csv('i_meta_category.csv', index=False)

## PART 10

In [24]:
import urllib.request
from urllib.error import URLError
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

j_food_lists = []

for url in tqdm(ji, desc="Crawling Progress"):
    if url:
        j_food_list = []

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            req = urllib.request.Request(url, headers=headers)
            code = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(code, 'html.parser')
            tag_cont_element = soup.find('ul', class_='tag_cont')
            time.sleep(1)
            if tag_cont_element:
                li_tags = tag_cont_element.find_all('li')
                for li in li_tags:
                    a_tag = li.find('a')
                    if a_tag:
                        name = a_tag.get_text()
                        j_food_list.append(name)
                    else:
                        j_food_list.append(np.nan)
                j_food_lists.append(i_food_list)
            else:
                j_food_lists.append([])
        except URLError as e:
            print(f"An error occurred while fetching URL: {url}\nError: {e}")
            j_food_lists.append([])
    else:
        j_food_lists.append([])

Crawling Progress:   5%|██▋                                                       | 378/8312 [07:12<1:50:00,  1.20it/s]

An error occurred while fetching URL: https://www.10000recipe.com/recipe/list.html?q=&query=&cat1=39&cat2=14&cat3=27&cat4=60&fct=&order=reco&lastcate=cat1&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource=
Error: <urlopen error [Errno 2] No such file or directory>


Crawling Progress:  25%|██████████████▎                                          | 2078/8312 [54:33<2:01:58,  1.17s/it]

An error occurred while fetching URL: https://www.10000recipe.com/recipe/list.html?q=&query=&cat1=37&cat2=15&cat3=71&cat4=53&fct=&order=reco&lastcate=cat1&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource=
Error: <urlopen error [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다>


Crawling Progress:  71%|██████████████████████████████████████▉                | 5890/8312 [2:11:50<1:06:09,  1.64s/it]

An error occurred while fetching URL: https://www.10000recipe.com/recipe/list.html?q=&query=&cat1=11&cat2=15&cat3=71&cat4=58&fct=&order=reco&lastcate=cat1&dsearch=&copyshot=&scrap=&degree=&portion=&time=&niresource=
Error: <urlopen error [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다>


Crawling Progress: 100%|█████████████████████████████████████████████████████████| 8312/8312 [3:00:21<00:00,  1.30s/it]


In [25]:
j_data = []
for i in range(len(j_food_lists)):
    li = '  '.join(j_food_lists[i])
    j_data.append(li)

print(j_data[0])

In [26]:
j_food_df = pd.DataFrame(j_data, columns = ['food_name'])
j_food_df.to_csv('j_meta_category.csv', index=False)